In [1]:
HTML(read(open("style.css"), String))

HTML{String}("<link href='http://fonts.googleapis.com/css?family=Alegreya+Sans:100,300,400,500,700,800,900,100italic,300italic,400italic,500italic,700italic,800italic,900italic' rel='stylesheet' type='text/css'>\r\n<link href='http://fonts.googleapis.com/css?family=Arvo:400,700,400italic' rel='stylesheet' type='text/css'>\r\n<link href='http://fonts.googleapis.com/css?family=PT+Mono' rel='stylesheet' type='text/css'>\r\n<link href='http://fonts.googleapis.com/css?family=Shadows+Into+Light' rel='stylesheet' type='text/css'>\r\n<link href='http://fonts.googleapis.com/css?family=Philosopher:400,700,400italic,700italic' rel='stylesheet' type='text/css'>\r\n\r\n<style>\r\n\r\n@font-face {\r\n    font-family: \"Computer Modern\";\r\n    src: url('http://mirrors.ctan.org/fonts/cm-unicode/fonts/otf/cmunss.otf');\r\n}\r\n\r\n.container { width: 100% }\r\n\r\n/* Formatting for header cells */\r\n.text_cell_render h1 {\r\n    font-family: 'Philosopher', sans-serif;\r\n    font-weight: 400;\r\n    font-size: 2.2em;\r\n    line-height: 100%;\r\n    color: rgb(0, 80, 120);\r\n    margin-bottom: 0.1em;\r\n    margin-top: 0.1em;\r\n    display: block;\r\n}\t\r\n.text_cell_render h2 {\r\n    font-family: 'Philosopher', serif;\r\n    font-weight: 400;\r\n    font-size: 1.9em;\r\n    line-height: 100%;\r\n    color: rgb(200,100,0);\r\n    margin-bottom: 0.1em;\r\n    margin-top: 0.1em;\r\n    display: block;\r\n}\t\r\n\r\n.text_cell_render h3 {\r\n    font-family: 'Philosopher', serif;\r\n    margin-top:12px;\r\n    margin-bottom: 3px;\r\n    font-style: italic;\r\n    color: rgb(94,127,192);\r\n}\r\n\r\n.text_cell_render h4 {\r\n    font-family: 'Philosopher', serif;\r\n}\r\n\r\n.text_cell_render h5 {\r\n    font-family: 'Alegreya Sans', sans-serif;\r\n    font-weight: 300;\r\n    font-size: 16pt;\r\n    color: grey;\r\n    font-style: italic;\r\n    margin-bottom: .1em;\r\n    margin-top: 0.1em;\r\n    display: block;\r\n}\r\n\r\n.text_cell_render h6 {\r\n    font-family: 'PT Mono', sans-serif;\r\n    font-weight: 300;\r\n    font-size: 10pt;\r\n    color: grey;\r\n    margin-bottom: 1px;\r\n    margin-top: 1px;\r\n}\r\n\r\n.text_cell_render em {\r\n    font-family: 'Philosopher', sans-serif;\r\n    color:        blue;\r\n    background-color: rgb(255,220,180);\r\n    font-size:    110%;\r\n    margin-left:   2px;\r\n    margin-right:  2px;\r\n    font-weight:   100;\r\n}\r\n\r\n.text_cell_render b {\r\n    color:            rgb(255,195,195);\r\n    background-color: rgb(0,0,0);\r\n    font-size:    110%;\r\n    margin-left:   2px;\r\n    margin-right:  2px;\r\n    font-weight:   650;\r\n}\r\n\r\n.text_cell_render tt {\r\n    font-size:    120%;\r\n    margin-left:   2px;\r\n    margin-right:  2px;\r\n    font-weight:   150;\r\n}\r\n\r\n.Codemirror {\r\n    font-family: \"PT Mono\";\r\n    font-size: 100%;\r\n}\r\n\r\n</style>\r\n\r\n")

In [2]:
# Pkg.add("NBInclude")
using NBInclude
# Pkg.add("Chess")
using Chess
using Test

In [23]:
@nbinclude("EvaluatePosition.ipynb")

terminal_evaluation (generic function with 1 method)

# Tests

## Test CalculatePosition for a single piece

1. Es soll getest werden zuerst eine Dame, ob sie korrekt bewertet wird
2. Danach sollen Bauern von getest werden, um zu überprüfen, ob die Reservese Funktion funktioniert

In [4]:
startBoard=startboard()
queen = Piece(WHITE, QUEEN)
wPawn = Piece(WHITE, PAWN)
bPawn = Piece(BLACK, PAWN)
sD6 = Square(FILE_D, RANK_6)
sD2 = Square(FILE_D, RANK_2)
sD7 = Square(FILE_D, RANK_7)
board = startboard()
domove!(board, "e4")
domove!(board, "d5")
sD5 = Square(FILE_D, RANK_5)
@testset "Get Value of Piece" begin
    @test calc_score(startBoard, queen, sD6) == 905
    @test calc_score(startBoard, wPawn, sD2) == 80 #100 pawn Value and -20 square 
    @test calc_score(startBoard, bPawn, sD7) == 80 #100 pawn Value and -20 square 
    @test calc_score(board, bPawn, sD5) == 120
end

Test Summary:      | Pass  Total  Time
Get Value of Piece |    4      4  1.8s


Test.DefaultTestSet("Get Value of Piece", Any[], 4, false, false, true, 1.677449675763e9, 1.677449677517e9)

## Evaluation of position 

In [5]:
b1 = fromfen("rnbqkbnr/ppp1pppp/8/3P4/8/8/PPPP1PPP/RNBQKBNR b KQkq - 0 1")
b2 = fromfen("rn1qkb1r/pppb1ppp/4pn2/3p4/3P1B2/2N5/PPPQPPPP/R3KBNR w KQkq - 2 5")
b3 = fromfen("1Q6/1R6/1B4k1/1N4p1/1P4n1/1K4b1/6r1/6q1 w - - 0 1")
display(b1)
display(b2)
@testset "Get Value of Piece" begin
    @test evaluate_position(b1) == 125
    @test evaluate_position(b2) == -5
    @test evaluate_position(b3) == 0
end

Board (rnbqkbnr/ppp1pppp/8/3P4/8/8/PPPP1PPP/RNBQKBNR b KQkq -):
 r  n  b  q  k  b  n  r 
 p  p  p  -  p  p  p  p 
 -  -  -  -  -  -  -  - 
 -  -  -  P  -  -  -  - 
 -  -  -  -  -  -  -  - 
 -  -  -  -  -  -  -  - 
 P  P  P  P  -  P  P  P 
 R  N  B  Q  K  B  N  R

Board (rn1qkb1r/pppb1ppp/4pn2/3p4/3P1B2/2N5/PPPQPPPP/R3KBNR w KQkq -):
 r  n  -  q  k  b  -  r 
 p  p  p  b  -  p  p  p 
 -  -  -  -  p  n  -  - 
 -  -  -  p  -  -  -  - 
 -  -  -  P  -  B  -  - 
 -  -  N  -  -  -  -  - 
 P  P  P  Q  P  P  P  P 
 R  -  -  -  K  B  N  R

Test Summary:      | Pass  Total  Time
Get Value of Piece |    3      3  0.9s


Test.DefaultTestSet("Get Value of Piece", Any[], 3, false, false, true, 1.677449682872e9, 1.677449683796e9)

## Test Capture Piece 

In [6]:
board = startboard()
domove!(board, "e4")
domove!(board, "d5")
display(board)
sD5 = Square(FILE_D,RANK_5)
sA8 = Square(FILE_A,RANK_8)
b2 = fromfen("rnbqkb1r/pP2pp1p/5np1/8/8/8/PPPP1PPP/RNBQKBNR w KQkq - 1 5")
@testset "Capture Piece" begin
    @test subCapturePiece(board, sD5) == -120
    @test subCapturePiece(b2, sA8) == -500
end

Board (rnbqkbnr/ppp1pppp/8/3p4/4P3/8/PPPP1PPP/RNBQKBNR w KQkq -):
 r  n  b  q  k  b  n  r 
 p  p  p  -  p  p  p  p 
 -  -  -  -  -  -  -  - 
 -  -  -  p  -  -  -  - 
 -  -  -  -  P  -  -  - 
 -  -  -  -  -  -  -  - 
 P  P  P  P  -  P  P  P 
 R  N  B  Q  K  B  N  R

Capture Piece: Error During Test at In[6]:9
  Test threw exception
  Expression: subCapturePiece(board, sD5) == -120
  UndefVarError: subCapturePiece not defined
  Stacktrace:
   [1] macro expansion
     @ C:\Users\yiwen\AppData\Local\Programs\Julia-1.8.2\share\julia\stdlib\v1.8\Test\src\Test.jl:464 [inlined]
   [2] macro expansion
     @ In[6]:9 [inlined]
   [3] macro expansion
     @ C:\Users\yiwen\AppData\Local\Programs\Julia-1.8.2\share\julia\stdlib\v1.8\Test\src\Test.jl:1357 [inlined]
   [4] top-level scope
     @ In[6]:9
Capture Piece: Error During Test at In[6]:10
  Test threw exception
  Expression: subCapturePiece(b2, sA8) == -500
  UndefVarError: subCapturePiece not defined
  Stacktrace:
   [1] macro expansion
     @ C:\Users\yiwen\AppData\Local\Programs\Julia-1.8.2\share\julia\stdlib\v1.8\Test\src\Test.jl:464 [inlined]
   [2] macro expansion
     @ In[6]:10 [inlined]
   [3] macro expansion
     @ C:\Users\yiwen\AppData\Local\Programs\Julia-1.8.2\share\julia\stdlib\v1.8\Test\

LoadError: [91mSome tests did not pass: 0 passed, 0 failed, 2 errored, 0 broken.[39m

## Test En Passant

In [7]:
board=fromfen("rnbqkbnr/ppp1pppp/8/3pP3/8/8/PPPP1PPP/RNBQKBNR b KQkq - 0 2")
domove!(board, "f5")
display(board)
sF6 = Square(FILE_F,RANK_6)
@testset "checkEnPassant" begin
    @test checkEnPassant(board, sF6) == -100
end

Board (rnbqkbnr/ppp1p1pp/8/3pPp2/8/8/PPPP1PPP/RNBQKBNR w KQkq f6):
 r  n  b  q  k  b  n  r 
 p  p  p  -  p  -  p  p 
 -  -  -  -  -  -  -  - 
 -  -  -  p  P  p  -  - 
 -  -  -  -  -  -  -  - 
 -  -  -  -  -  -  -  - 
 P  P  P  P  -  P  P  P 
 R  N  B  Q  K  B  N  R

checkEnPassant: Test Failed at In[7]:6
  Expression: checkEnPassant(board, sF6) == -100
   Evaluated: 100 == -100
Stacktrace:
 [1] macro expansion
   @ C:\Users\yiwen\AppData\Local\Programs\Julia-1.8.2\share\julia\stdlib\v1.8\Test\src\Test.jl:464 [inlined]
 [2] macro expansion
   @ In[7]:6 [inlined]
 [3] macro expansion
   @ C:\Users\yiwen\AppData\Local\Programs\Julia-1.8.2\share\julia\stdlib\v1.8\Test\src\Test.jl:1357 [inlined]
 [4] top-level scope
   @ In[7]:6
Test Summary:  | Fail  Total  Time
checkEnPassant |    1      1  3.3s


LoadError: [91mSome tests did not pass: 0 passed, 1 failed, 0 errored, 0 broken.[39m

In [8]:
board=fromfen("rn1qkb1r/pppb1ppp/4pn2/3p4/3P1B2/2N5/PPPQPPPP/R3KBNR w KQkq - 2 5")
sC1 = Square(FILE_C,RANK_1)
piece = Piece(WHITE, KING)
@testset "calc_castle" begin
    @test calc_castle(board, piece ,sC1) == 5
end

Test Summary: | Pass  Total  Time
calc_castle   |    1      1  0.0s


Test.DefaultTestSet("calc_castle", Any[], 1, false, false, true, 1.677449691346e9, 1.677449691366e9)

## Testing IsEndGame


In [9]:
@testset "Testing Endgame" begin
    @test is_endgame(fromfen("2bqk3/8/8/8/8/8/5B2/3QK3 w - - 0 1")) == true
    @test is_endgame(fromfen("rnb1kbnr/ppp2ppp/8/8/8/8/PPP2PPP/RNB1KBNR w KQkq - 0 1")) == true
    @test is_endgame(fromfen("2b1kb2/pp4pp/8/8/8/8/PP4P1/R2QK3 w Q - 0 1")) == false
end

Test Summary:   | Pass  Total  Time
Testing Endgame |    3      3  0.0s


Test.DefaultTestSet("Testing Endgame", Any[], 3, false, false, true, 1.677449691397e9, 1.677449691417e9)

## Evaluate Move for terminal positions

In [21]:
b1 = fromfen("7k/8/8/3K4/8/6R1/8/5R2 w - - 0 1")
m1 = movefromstring("f1h1")
display(b1)

b2 = fromfen("k7/7K/8/8/6r1/8/8/5r2 b - - 0 1")
m2 = movefromstring("f1h1")
display(b2)
println(evaluate_move(b2, m2, evaluate_position(b2)))
@testset "Evaluate Move terminal position" begin
    @test evaluate_move(b1, m1, evaluate_position(b1)) == evaluate_position(domove(b1, m1))
    @test evaluate_move(b2, m2, evaluate_position(b2)) == evaluate_position(domove(b2, m2))
end

Board (7k/8/8/3K4/8/6R1/8/5R2 w - -):
 -  -  -  -  -  -  -  k 
 -  -  -  -  -  -  -  - 
 -  -  -  -  -  -  -  - 
 -  -  -  K  -  -  -  - 
 -  -  -  -  -  -  -  - 
 -  -  -  -  -  -  R  - 
 -  -  -  -  -  -  -  - 
 -  -  -  -  -  R  -  -

Board (k7/7K/8/8/6r1/8/8/5r2 b - -):
 k  -  -  -  -  -  -  - 
 -  -  -  -  -  -  -  K 
 -  -  -  -  -  -  -  - 
 -  -  -  -  -  -  -  - 
 -  -  -  -  -  -  r  - 
 -  -  -  -  -  -  -  - 
 -  -  -  -  -  -  -  - 
 -  -  -  -  -  r  -  -

-100000
Test Summary:                   | Pass  Total  Time
Evaluate Move terminal position |    2      2  0.0s


Test.DefaultTestSet("Evaluate Move terminal position", Any[], 2, false, false, true, 1.677450006836e9, 1.677450006837e9)

## Evaluate Move for endgame transition

In [24]:
b1 = fromfen("r1b1kbnr/p2p1ppp/2pP4/1p2p1B1/4P3/1B3q2/PPP2PPP/RN3RK1 w kq - 0 1")
m1 = movefromstring("g2f3")

display(b1)
display(domove(b1, m1))

@testset "evaluate_move" begin
    @test evaluate_move(b1, m1, evaluate_position(b1)) == evaluate_position(domove(b1, m1)) # regular position
end

Board (r1b1kbnr/p2p1ppp/2pP4/1p2p1B1/4P3/1B3q2/PPP2PPP/RN3RK1 w kq -):
 r  -  b  -  k  b  n  r 
 p  -  -  p  -  p  p  p 
 -  -  p  P  -  -  -  - 
 -  p  -  -  p  -  B  - 
 -  -  -  -  P  -  -  - 
 -  B  -  -  -  q  -  - 
 P  P  P  -  -  P  P  P 
 R  N  -  -  -  R  K  -

Board (r1b1kbnr/p2p1ppp/2pP4/1p2p1B1/4P3/1B3P2/PPP2P1P/RN3RK1 b kq -):
 r  -  b  -  k  b  n  r 
 p  -  -  p  -  p  p  p 
 -  -  p  P  -  -  -  - 
 -  p  -  -  p  -  B  - 
 -  -  -  -  P  -  -  - 
 -  B  -  -  -  P  -  - 
 P  P  P  -  -  P  -  P 
 R  N  -  -  -  R  K  -

Endgame transition
125
SQ_E8
-125
-155
SQ_G1
125
95
Test Summary: | Pass  Total  Time
evaluate_move |    1      1  0.3s


Test.DefaultTestSet("evaluate_move", Any[], 1, false, false, true, 1.677450258448e9, 1.677450258751e9)

## Evaluate Move

In [12]:
# white to move
b1 = startboard()
m1 = Move(Square(FILE_E,RANK_2), Square(FILE_E,RANK_4))

# Capture white and promotion
b2 = fromfen("rnbqkb1r/pP2pp1p/5np1/8/8/8/PPPP1PPP/RNBQKBNR w KQkq - 1 5")
m2 = Move(Square(FILE_B,RANK_7), Square(FILE_A,RANK_8), QUEEN)

# black to move
b3 = fromfen("7k/P7/8/8/8/8/8/K7 b - - 0 1")
m3 = movefromstring("h8h7")
b4 = fromfen("rnbqkbnr/ppp2ppp/8/3Pp3/8/5N2/PPPP1PPP/RNBQKB1R b KQkq - 0 1")
m4 = movefromstring("d8h4")

# Captures white
b5 = fromfen("rnb1kbnr/ppp2ppp/8/3Pp3/7q/5N2/PPPP1PPP/RNBQKB1R w KQkq - 0 1")
m5 = movefromstring("f3h4")

# show boards
display(domove(b1, m1))
display(domove(b2, m2))
display(domove(b3, m3))
display(domove(b4, m4))
display(domove(b5, m5))

@testset "evaluate_move" begin
    @test evaluate_move(b1, m1, evaluate_position(b1)) == evaluate_position(domove(b1, m1)) # regular position
    @test evaluate_move(b2, m2, evaluate_position(b2)) == evaluate_position(domove(b2, m2))
    @test evaluate_move(b3, m3, evaluate_position(b3)) == evaluate_position(domove(b3, m3)) # black king movement
    @test evaluate_move(b4, m4, evaluate_position(b4)) == evaluate_position(domove(b4, m4)) # Queen blunder black
    @test evaluate_move(b5, m5, evaluate_position(b5)) == evaluate_position(domove(b5, m5))
end

println(evaluate_position(b4))
println(evaluate_position(domove(b4, m4)))

Board (rnbqkbnr/pppppppp/8/8/4P3/8/PPPP1PPP/RNBQKBNR b KQkq -):
 r  n  b  q  k  b  n  r 
 p  p  p  p  p  p  p  p 
 -  -  -  -  -  -  -  - 
 -  -  -  -  -  -  -  - 
 -  -  -  -  P  -  -  - 
 -  -  -  -  -  -  -  - 
 P  P  P  P  -  P  P  P 
 R  N  B  Q  K  B  N  R

Board (Qnbqkb1r/p3pp1p/5np1/8/8/8/PPPP1PPP/RNBQKBNR b KQk -):
 Q  n  b  q  k  b  -  r 
 p  -  -  -  p  p  -  p 
 -  -  -  -  -  n  p  - 
 -  -  -  -  -  -  -  - 
 -  -  -  -  -  -  -  - 
 -  -  -  -  -  -  -  - 
 P  P  P  P  -  P  P  P 
 R  N  B  Q  K  B  N  R

Board (8/P6k/8/8/8/8/8/K7 w - -):
 -  -  -  -  -  -  -  - 
 P  -  -  -  -  -  -  k 
 -  -  -  -  -  -  -  - 
 -  -  -  -  -  -  -  - 
 -  -  -  -  -  -  -  - 
 -  -  -  -  -  -  -  - 
 -  -  -  -  -  -  -  - 
 K  -  -  -  -  -  -  -

Board (rnb1kbnr/ppp2ppp/8/3Pp3/7q/5N2/PPPP1PPP/RNBQKB1R w KQkq -):
 r  n  b  -  k  b  n  r 
 p  p  p  -  -  p  p  p 
 -  -  -  -  -  -  -  - 
 -  -  -  P  p  -  -  - 
 -  -  -  -  -  -  -  q 
 -  -  -  -  -  N  -  - 
 P  P  P  P  -  P  P  P 
 R  N  B  Q  K  B  -  R

Board (rnb1kbnr/ppp2ppp/8/3Pp3/7N/8/PPPP1PPP/RNBQKB1R b KQkq -):
 r  n  b  -  k  b  n  r 
 p  p  p  -  -  p  p  p 
 -  -  -  -  -  -  -  - 
 -  -  -  P  p  -  -  - 
 -  -  -  -  -  -  -  N 
 -  -  -  -  -  -  -  - 
 P  P  P  P  -  P  P  P 
 R  N  B  Q  K  B  -  R

Test Summary: | Pass  Total  Time
evaluate_move |    5      5  0.0s
135
135


### Evaluate move - Promotion

In [13]:
bw = fromfen("8/pk2P3/8/8/8/8/PK6/8 w - - 0 1")
display(bw)
currentscore_w = evaluate_position(bw)
print("Static evaluation of Position: $(currentscore_w)")
mwN = movefromstring("e7e8N")    # mwN = move white kNight
mwB = movefromstring("e7e8B")
mwR = movefromstring("e7e8R")
mwQ = movefromstring("e7e8Q")

bb = fromfen("8/pk6/8/8/8/8/PK2p3/8 b - - 0 1")
display(bb)
currentscore_b = evaluate_position(bb)
print("Static evaluation of Position: $(currentscore_b)")
mbN = movefromstring("e2e1N")
mbB = movefromstring("e2e1B")
mbR = movefromstring("e2e1R")
mbQ = movefromstring("e2e1Q")

@testset "evaluate_move_position" begin
    @test evaluate_move(bw, mwN, currentscore_w) ==  290 # promotion white Knight
    @test evaluate_move(bw, mwB, currentscore_w) ==  320 # promotion white Bishop
    @test evaluate_move(bw, mwR, currentscore_w) ==  500 # promotion white Rook
    @test evaluate_move(bw, mwQ, currentscore_w) ==  895 # promotion white Queen
    
    @test evaluate_move(bb, mbN, currentscore_b) ==  -290 # promotion black Knight
    @test evaluate_move(bb, mbB, currentscore_b) ==  -320 # promotion black Bishop
    @test evaluate_move(bb, mbR, currentscore_b) ==  -500 # promotion black Rook
    @test evaluate_move(bb, mbQ, currentscore_b) ==  -895 # promotion black Queen
end

Board (8/pk2P3/8/8/8/8/PK6/8 w - -):
 -  -  -  -  -  -  -  - 
 p  k  -  -  P  -  -  - 
 -  -  -  -  -  -  -  - 
 -  -  -  -  -  -  -  - 
 -  -  -  -  -  -  -  - 
 -  -  -  -  -  -  -  - 
 P  K  -  -  -  -  -  - 
 -  -  -  -  -  -  -  -

Static evaluation of Position: 150

Board (8/pk6/8/8/8/8/PK2p3/8 b - -):
 -  -  -  -  -  -  -  - 
 p  k  -  -  -  -  -  - 
 -  -  -  -  -  -  -  - 
 -  -  -  -  -  -  -  - 
 -  -  -  -  -  -  -  - 
 -  -  -  -  -  -  -  - 
 P  K  -  -  p  -  -  - 
 -  -  -  -  -  -  -  -

Static evaluation of Position: -150Test Summary:          | Pass  Total  Time
evaluate_move_position |    8      8  0.0s


Test.DefaultTestSet("evaluate_move_position", Any[], 8, false, false, true, 1.677449692385e9, 1.677449692385e9)